In [1]:
import pandas as pd
import evalml

In [2]:
df=pd.read_csv('clean_laptop_data1.csv')

In [3]:
df.head()

,Unnamed: 0,Company,TypeName,Ram,Weight,Price,TouchScreen,IPS,ppi,Cpu_brand,HDD,SSD,Gpu_brand,os
0,0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,0,256,Intel,Others/No OS/Linux
3,3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,0,256,Intel,Mac


In [4]:
df=df.drop(df.columns[[0]], axis=1)

In [5]:
X=df.drop("Price", axis='columns')
Y=df['Price']

In [6]:
X_train, X_test, Y_train, Y_test=evalml.preprocessing.split_data(X, Y, problem_type='regression')


In [7]:
from evalml.automl import AutoMLSearch

In [8]:
automl =AutoMLSearch(X_train=X_train, y_train=Y_train, problem_type='regression')

In [9]:
automl.search()

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.001e+11, tolerance: 1.005e+08
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.024e+11, tolerance: 8.941e+07
Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.383e+10, tolerance: 9.251e+07


{1: {'Random Forest Regressor w/ Imputer + One Hot Encoder': 1.0649285316467285,
  'Total time of batch': 1.1904845237731934},
 2: {'Random Forest Regressor w/ Imputer + One Hot Encoder + RF Regressor Select From Model': 1.3902225494384766,
  'Total time of batch': 1.5212860107421875},
 3: {'Decision Tree Regressor w/ Select Columns By Type Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Imputer + One Hot Encoder': 0.4859464168548584,
  'Extra Trees Regressor w/ Select Columns By Type Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Imputer + One Hot Encoder': 0.8992152214050293,
  'XGBoost Regressor w/ Select Columns By Type Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Imputer + One Hot Encoder': 1.9184598922729492,
  'CatBoost Regressor w/ Select Columns By Type Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Imputer': 1.658822774887085,
  'LightGB

In [10]:
automl.rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,5,XGBoost Regressor w/ Select Columns By Type Tr...,5,0.812658,0.812658,0.024208,30647.329661,False,{'Numeric Pipeline - Select Columns By Type Tr...
1,2,Random Forest Regressor w/ Imputer + One Hot E...,2,0.776051,0.776051,0.037208,29271.274533,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,1,Random Forest Regressor w/ Imputer + One Hot E...,1,0.771325,0.771325,0.031907,29093.648936,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,8,Elastic Net Regressor w/ Select Columns By Typ...,8,0.769673,0.769673,0.025587,29031.527054,False,{'Numeric Pipeline - Select Columns By Type Tr...
4,4,Extra Trees Regressor w/ Select Columns By Typ...,4,0.750409,0.750409,0.056028,28307.414624,False,{'Numeric Pipeline - Select Columns By Type Tr...
5,7,LightGBM Regressor w/ Select Columns By Type T...,7,0.733964,0.733964,0.032808,27689.262795,False,{'Numeric Pipeline - Select Columns By Type Tr...
6,3,Decision Tree Regressor w/ Select Columns By T...,3,0.676876,0.676876,0.060197,25543.354668,False,{'Numeric Pipeline - Select Columns By Type Tr...
7,6,CatBoost Regressor w/ Select Columns By Type T...,6,0.254636,0.254636,0.025954,9671.603358,False,{'Numeric Pipeline - Select Columns By Type Tr...
8,0,Mean Baseline Regression Pipeline,0,-0.002660,-0.002660,0.002205,0.000000,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [11]:
best_pipeline= automl.best_pipeline

In [12]:
best_pipeline

pipeline = RegressionPipeline(component_graph={'Numeric Pipeline - Select Columns By Type Transformer': ['Select Columns By Type Transformer', 'X', 'y'], 'Numeric Pipeline - Imputer': ['Imputer', 'Numeric Pipeline - Select Columns By Type Transformer.x', 'y'], 'Numeric Pipeline - Select Columns Transformer': ['Select Columns Transformer', 'Numeric Pipeline - Imputer.x', 'y'], 'Categorical Pipeline - Select Columns Transformer': ['Select Columns Transformer', 'X', 'y'], 'Categorical Pipeline - Imputer': ['Imputer', 'Categorical Pipeline - Select Columns Transformer.x', 'y'], 'Categorical Pipeline - One Hot Encoder': ['One Hot Encoder', 'Categorical Pipeline - Imputer.x', 'y'], 'XGBoost Regressor': ['XGBoost Regressor', 'Numeric Pipeline - Select Columns Transformer.x', 'Categorical Pipeline - One Hot Encoder.x', 'y']}, parameters={'Numeric Pipeline - Select Columns By Type Transformer':{'column_types': ['category', 'EmailAddress', 'URL'], 'exclude': True}, 'Numeric Pipeline - Imputer':{

In [13]:
automl.describe_pipeline(automl.rankings.iloc[0]['id'])


***********************************************************************************************************************************************************
* XGBoost Regressor w/ Select Columns By Type Transformer + Imputer + Select Columns Transformer + Select Columns Transformer + Imputer + One Hot Encoder *
***********************************************************************************************************************************************************

Problem Type: regression
Model Family: XGBoost

Pipeline Steps
1. Select Columns By Type Transformer
	 * column_types : ['category', 'EmailAddress', 'URL']
	 * exclude : True
2. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * boolean_impute_strategy : most_frequent
	 * categorical_fill_value : None
	 * numeric_fill_value : None
	 * boolean_fill_value : None
3. Select Columns Transformer
	 * columns : ['Ram', 'Weight', 'TouchScreen', 'IPS', 'ppi', 'HDD', 'SSD']
4. Select Columns 

In [14]:
### Evaluate on hold out data
best_pipeline.score(X_test, Y_test, objectives=['r2'])

OrderedDict([('R2', 0.7720827236172391)])

In [15]:
import pickle
pickle.dump(best_pipeline,open('evalpipe.pkl','wb'))